In [1]:
import requests
import geopy.distance
import pandas as pd
from itertools import chain
import requests
import datetime
import os
from toolz import pipe

url = "https://www.airbnb.com/api/v3/ExploreSections"

querystring = {"operationName":"ExploreSections",
               "locale":"en",
               "currency":"USD",
               "variables":"{\"isInitialLoad\":true,\"hasLoggedIn\":true,\"cdnCacheSafe\":false,\"source\":\"EXPLORE\",\"exploreRequest\":{\"metadataOnly\":false,\"version\":\"1.8.3\",\"placeId\":\"ChIJW8vuDrriaocRRu1CVaEuuW8\",\"checkin\":\"2022-10-02\",\"checkout\":\"2022-10-03\",\"datePickerType\":\"calendar\",\"adults\":2,\"searchType\":\"unknown\",\"tabId\":\"home_tab\",\"flexibleTripLengths\":[\"one_week\"],\"priceFilterNumNights\":1,\"refinementPaths\":[\"/homes\"],\"federatedSearchSessionId\":\"85b528f4-8e63-4b2c-a35a-ebba07489c1d\",\"itemsOffset\":0,\"sectionOffset\":2,\"query\":\"Jefferson, Colorado, United States\",\"itemsPerGrid\":20,\"cdnCacheSafe\":false,\"treatmentFlags\":[\"flex_destinations_june_2021_launch_web_treatment\",\"new_filter_bar_v2_fm_header\",\"merch_header_breakpoint_expansion_web\",\"flexible_dates_12_month_lead_time\",\"storefronts_nov23_2021_homepage_web_treatment\",\"lazy_load_flex_search_map_compact\",\"lazy_load_flex_search_map_wide\",\"im_flexible_may_2022_treatment\",\"im_flexible_may_2022_treatment\",\"flex_v2_review_counts_treatment\",\"flexible_dates_options_extend_one_three_seven_days\",\"super_date_flexibility\",\"micro_flex_improvements\",\"micro_flex_show_by_default\",\"search_input_placeholder_phrases\",\"pets_fee_treatment\"],\"screenSize\":\"large\",\"isInitialLoad\":true,\"hasLoggedIn\":true}}","extensions":"{\"persistedQuery\":{\"version\":1,\"sha256Hash\":\"ef48c5cbc4059de87060d37d4c62b72ad708e5ffba74fb8f042c970f0664f0c5\"}}"}

headers = {
    "authority": "www.airbnb.com",
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9",
    "content-type": "application/json",
    "referer": "https://www.airbnb.com/s/Jefferson--Colorado--United-States/homes?flexible_trip_dates%5B%5D=october&date_picker_type=flexible_dates&search_type=unknown&tab_id=home_tab&adults=2&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=weekend_trip&federated_search_session_id=bbcfac66-2fe4-40f1-a432-b03d9eccf31a&pagination_search=true&items_offset=0&section_offset=3",
    "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.61 Safari/537.36",
    "x-airbnb-api-key": "d306zoyjsyarp7ifhu67rjxn52tv0t20",
    "x-airbnb-graphql-platform": "web",
    "x-airbnb-graphql-platform-client": "minimalist-niobe",
    "x-airbnb-supports-airlock-v2": "true",
    "x-csrf-without-token": "1",
    "x-niobe-short-circuited": "true"
}

In [2]:
def var_edit(checkin, checkout, itemindex):
    """
    format yyyy-mm-dd
    """
    return '{\"isInitialLoad\":true,\"hasLoggedIn\":true,\"cdnCacheSafe\":false,\"source\":\"EXPLORE\",\"exploreRequest\":{\"metadataOnly\":false,\"version\":\"1.8.3\",\"placeId\":\"ChIJW8vuDrriaocRRu1CVaEuuW8\",\"checkin\":\"' +\
    checkin + '\",\"checkout\":\"' +\
    checkout + '\",\"datePickerType\":\"calendar\",\"adults\":2,\"searchType\":\"unknown\",\"tabId\":\"home_tab\",\"flexibleTripLengths\":[\"one_week\"],\"priceFilterNumNights\":1,\"refinementPaths\":[\"/homes\"],\"federatedSearchSessionId\":\"85b528f4-8e63-4b2c-a35a-ebba07489c1d\",\"itemsOffset\":' +\
    str(itemindex) + ',\"sectionOffset\":2,\"query\":\"Jefferson, Colorado, United States\",\"itemsPerGrid\":20,\"cdnCacheSafe\":false,\"treatmentFlags\":[\"flex_destinations_june_2021_launch_web_treatment\",\"new_filter_bar_v2_fm_header\",\"merch_header_breakpoint_expansion_web\",\"flexible_dates_12_month_lead_time\",\"storefronts_nov23_2021_homepage_web_treatment\",\"lazy_load_flex_search_map_compact\",\"lazy_load_flex_search_map_wide\",\"im_flexible_may_2022_treatment\",\"im_flexible_may_2022_treatment\",\"flex_v2_review_counts_treatment\",\"flexible_dates_options_extend_one_three_seven_days\",\"super_date_flexibility\",\"micro_flex_improvements\",\"micro_flex_show_by_default\",\"search_input_placeholder_phrases\",\"pets_fee_treatment\"],\"screenSize\":\"large\",\"isInitialLoad\":true,\"hasLoggedIn\":true}}","extensions":"{\"persistedQuery\":{\"version\":1,\"sha256Hash\":\"ef48c5cbc4059de87060d37d4c62b72ad708e5ffba74fb8f042c970f0664f0c5\"}'


In [3]:
def pull_cost_breakdown(pricedetailsobj):
    try:
        price_dict = pricedetailsobj[0]
        return ' | '.join([
            f"{key['description']}: {key['priceString']}" for key in price_dict['items']]
        )
    except:
        return 'unable to parse'

In [4]:
items = list(range(0, 250, 20))


In [25]:

checkin_date = '2022-11-01'
checkout_date = '2022-11-10'


In [6]:
relevant_columns = [
        'name',
        'distanceFromTarryallCabin',
        'avgRating',
        'reviewsCount',
        'isNewListing',
        'isSuperhost',
        'amenities',
        'structuredStayDisplayPrice.primaryLine.accessibilityLabel',
        'costBreakdown',
        'title',
        'personCapacity',
        'availableBeds',
        'start_date',
        'timeBooked',
        'distanceFromTarryallCabin_w',

    ]

In [28]:
def parse_bnb_info(df):
    return df.assign(
            **{
                'availableBeds': df.loc[:, 'structuredContent.primaryLine'].apply(
                    lambda x: ';'.join([m['body'] for m in x])
                ),
                'costBreakdown': df.loc[:, 'structuredStayDisplayPrice.explanationData.priceDetails'].apply(
                    pull_cost_breakdown
                ),
                'coords': df.apply(
                    lambda x: (x.lat, x.lng),
                    axis=1
                ),
                'amenities': df.loc[:, 'previewAmenityNames'].apply(
                    lambda listing_list:
                    ' | '.join(listing_list)
                )
            }
        )


def calc_distance_cols(df):
    #  use as static metric in below calc
    tarryall = (39.296114, -105.7186119)
    
    df = df.assign(
            **{
                'distanceFromTarryallCabin_w': df.coords.apply(
                    lambda coordinates:
                    geopy.distance.geodesic(tarryall, coordinates).mi
                ),
            }
        )
    return df.assign(
            **{
                'distanceFromTarryallCabin': df.distanceFromTarryallCabin_w.apply(
                    lambda coordinates:
                    f'{coordinates} miles'
                ),

            }
        )


def total_stays_for_time(checkin, checkout, items):
    
    rel_cols = relevant_columns
    
    # set up all the requests to the api for 20-items-per-page approach

    final_res = list()
    for item in items:
        querystring['variables'] = var_edit(checkin, checkout, item)
        final_res.append(
            requests.request("GET", url, headers=headers, params=querystring)
        )

    #  remove failed requests
    final_res = list(
        filter(
            None,
            final_res
        )
    )

    #  drilldown to relevant section in airbnb (as of Aug2022)
    json_total_data = list(
        map(
            lambda response:
            response.json()['data']['presentation']['explore']['sections']['sections'][2]['section']['child']['section']['items'],
            final_res
        )
    )

    #  some vals are getting mixed, store to QA
    backup_json_total = json_total_data

    #  update/consolidate information into a single dict
    list(
        chain(
            *list(
                map(
                    lambda stays_chunk:
                        map(
                        lambda stay:
                            # stay['listing'].update(stay['pricingQuote']['structuredStayDisplayPrice']),
                            stay['listing'].update(stay['pricingQuote']),
                            stays_chunk
                    ),
                    json_total_data
                )
            )
        )
    )


    #  merge list of lists for ease of use
    json_total_listings = list(
        chain(
            *json_total_data
        )
    )
    

    df = pipe(
        pd.concat(  
            list(
                map(
                    lambda stay:
                        pd.json_normalize(
                            stay['listing']
                        ),
                    json_total_listings
                )
            )
        ).assign(
            **{
                'timeBooked': f'{checkin}:{checkout}',
                'start_date': pd.to_datetime(checkin),
            }
        ),
        parse_bnb_info,
        calc_distance_cols,
        lambda all_listings_in_jefferson:
        all_listings_in_jefferson[rel_cols].drop_duplicates(
        ).reset_index(
        ).drop(
            columns='index'
        ),
    )
    return df.sort_values(
            'distanceFromTarryallCabin_w'
        )


In [8]:
# df = total_stays_for_time(checkin_date, checkout_date, items)
# df

In [9]:
def calc_time(start, end, daysbetween):
    time_iters = list(range(0, (end-start).days, daysbetween))
    dates = []
    for i in range(len(time_iters)):
        try: 
            dates.append(
                (
                    str(start + datetime.timedelta(days=time_iters[i])),
                    str(start + datetime.timedelta(days=time_iters[i+1]))
                )
            )
        except IndexError:
            continue
    return dates

In [10]:
start = datetime.date(2022,11,1)
end = datetime.date(2022,12,1)

iterations = list(range(0, 40, 20))
two_dates = calc_time(start, end, 2)
weekly_dates = calc_time(start, end, 7)


In [11]:
weekly_dates

[('2022-11-01', '2022-11-08'),
 ('2022-11-08', '2022-11-15'),
 ('2022-11-15', '2022-11-22'),
 ('2022-11-22', '2022-11-29')]

In [12]:
len(two_dates) * len(iterations), len(weekly_dates) * len(iterations)

(28, 8)

In [30]:
two_days_booking = pd.concat(
    list(
        map(
            lambda date:
            total_stays_for_time(*date, iterations),
            two_dates
        )
    )
)


In [31]:
weekly_booking = pd.concat(
    list(
        map(
            lambda date:
            total_stays_for_time(*date, iterations),
            weekly_dates
        )
    )
)

In [32]:
two_days_booking.drop_duplicates(inplace=True)
weekly_booking.drop_duplicates(inplace=True)

twod_nodup = two_days_booking.drop_duplicates(['name', 'costBreakdown'])
weekly_nodup = weekly_booking.drop_duplicates(['name', 'costBreakdown'])

In [33]:
twod_nodup[~(twod_nodup.loc[:, 'costBreakdown'] == 'unable to parse')].shape, twod_nodup.shape

((267, 15), (267, 15))

In [34]:
writer = pd.ExcelWriter(
    os.path.join(
        os.getcwd(),
        r'airbnb_info/allListingsNovember.xlsx'
    ), 
    engine='openpyxl'
)

twod_nodup[~(twod_nodup.loc[:, 'costBreakdown'] == 'unable to parse')].sort_values(
    ['distanceFromTarryallCabin', 'name']
).to_excel(writer, sheet_name='two_day_bookings_noDuplicates', index=False)

two_days_booking[~(two_days_booking.loc[:, 'costBreakdown'] == 'unable to parse')].sort_values(
    ['distanceFromTarryallCabin', 'name']
).to_excel(writer, sheet_name='two_day_bookings_All', index=False)

weekly_booking[~(weekly_booking.loc[:, 'costBreakdown'] == 'unable to parse')].sort_values(
    ['distanceFromTarryallCabin', 'name']
).to_excel(writer, sheet_name='Weekly_bookings_All', index=False)

weekly_nodup[~(weekly_nodup.loc[:, 'costBreakdown'] == 'unable to parse')].sort_values(
    ['distanceFromTarryallCabin', 'name']
).to_excel(writer, sheet_name='Weekly_bookings_noDuplicates', index=False)

writer.save()

In [20]:
two_days_booking.start_date

28   2022-11-01
12   2022-11-01
29   2022-11-01
34   2022-11-01
31   2022-11-01
        ...    
32   2022-11-01
24   2022-11-01
34   2022-11-01
33   2022-11-01
25   2022-11-01
Name: start_date, Length: 292, dtype: datetime64[ns]